<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)">Mixed-data neural network</h2>


This notebook applies a mixed data model architecture to a dataset of satellite images of areas where the most serious (serious and fatal) accidents occurred, in order to predict the locations of the most serious traffic accidents. [Source](https://heartbeat.fritz.ai/building-a-mixed-data-neural-network-in-keras-to-predict-accident-locations-d51a63b738cf)

In [1]:
import glob
import os
import cv2
import datetime
import math
import urllib
import argparse
import locale
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 50)

from scipy.spatial import cKDTree
from shapely.ops import nearest_points
import geopandas

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from IPython.display import SVG

%matplotlib inline
np.random.seed(123)

In [2]:
print(tf.__version__)

2.3.1


In [3]:
accidents = pd.read_csv('../../../data/London_accidents_merged.csv')
accidents.head()

,Accident_Index,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,1st_Road_Class,Road_Type,Speed_limit,Junction_Detail,2nd_Road_Class,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Hour,Two_Hour_Groupings,Time_of_Day,Was_Daylight,Was_Bad_Weather,Was_Road_Dry,log_Number_of_Casualties,log_Number_of_Vehicles,LSOA,population_per_hectare,bicycle_aadf,motorbike_aadf,car_aadf,bus_aadf,light_goods_vehicle_aadf,heavy_goods_vehicle_aadf,Road,RCat
0,201301BS70003,-0.171402,51.486361,Serious,2,1,2013-01-02,Wednesday,A,Single carriageway,30.0,T or staggered junction,Unclassified,Daylight,Fine no high winds,Dry,Urban,9,8am-10am,Morning,Yes,No,Yes,0.000000,0.693147,E01002844,110.8,1634.4,860.4,14888.0,1139.8,2297.0,352.0,A3217,PA
1,201301BS70005,-0.173356,51.495115,Slight,1,2,2013-01-04,Friday,A,Single carriageway,30.0,Crossroads,A,Daylight,Other,Dry,Urban,8,8am-10am,Morning,Yes,Yes,Yes,0.693147,0.000000,E01002821,74.6,559.6,1516.0,28505.6,1396.2,3868.6,1003.0,A4,PA
2,201301BS70006,-0.210767,51.518353,Slight,1,1,2013-01-07,Monday,B,Single carriageway,30.0,Crossroads,B,Daylight,Fine no high winds,Dry,Urban,11,10am-12pm,Office hours,Yes,No,Yes,0.000000,0.000000,E01002878,133.4,2.6,3898.2,63274.8,763.4,15253.6,3185.8,A40,PA
3,201301BS70007,-0.209675,51.516808,Slight,2,1,2013-01-10,Thursday,B,Single carriageway,30.0,Crossroads,C,Daylight,Fine no high winds,Dry,Urban,10,10am-12pm,Office hours,Yes,No,Yes,0.000000,0.693147,E01002831,179.2,2.6,3898.2,63274.8,763.4,15253.6,3185.8,A40,PA
4,201301BS70009,-0.194332,51.492922,Slight,2,1,2013-01-04,Friday,A,One way street,30.0,T or staggered junction,Unclassified,Darkness - lights lit,Fine no high winds,Dry,Urban,17,4pm-6pm,Rush hour,No,No,Yes,0.000000,0.693147,E01002851,272.3,869.2,1229.8,20478.6,897.2,4951.6,1251.4,A3220,PA


In [4]:
# This additional dataframe contains the latitude and longitude centre points for each LSOA
lsoa_latlong = pd.read_csv('../../../data/UK_LSOA_bounding_boxes.csv', usecols=['lsoa', 'Latitude', 'Longitude'])
lsoa_latlong.head()

,lsoa,Latitude,Longitude
0,E01000001,51.520269,-0.0950
1,E01000001,51.519848,-0.0967
2,E01000001,51.519030,-0.0962
3,E01000001,51.516904,-0.0981
4,E01000003,51.522376,-0.0973


In [7]:
population = pd.read_csv('../../../data/Population_density.csv')
population.head()

,LSOA,population_per_hectare
0,E01012334,0.4
1,E01012335,12.1
2,E01012366,0.3
3,E01033481,9.3
4,E01033482,6.9


In [8]:
traffic = pd.read_csv('../../../data/Traffic_averages.csv')
traffic.head()

,CP,S Ref Latitude,S Ref Longitude,bicycle_aadf,motorbike_aadf,car_aadf,bus_aadf,light_goods_vehicle_aadf,heavy_goods_vehicle_aadf
0,51,49.915023,-6.317073,238.2,96.8,539.4,27.0,379.0,40.4
1,52,49.912343,-6.305686,87.4,79.4,629.0,9.0,221.8,11.0
2,53,49.917141,-6.306114,181.8,142.6,777.2,32.2,403.8,29.4
3,54,49.917810,-6.298996,61.2,54.8,342.8,6.0,251.6,34.4
4,55,49.918585,-6.295094,33.6,20.8,165.6,0.0,150.8,15.2


In [9]:
# Combining the motor vehicle traffic columns, based on the multi-collinearity found in model 1
to_sum = ['motorbike_aadf', 'car_aadf', 'bus_aadf', 'light_goods_vehicle_aadf', 'heavy_goods_vehicle_aadf']
traffic['motor_vehicle_aadf'] = traffic[to_sum].sum(axis=1)
traffic.drop(to_sum, axis=1, inplace=True)
traffic.head()

,CP,S Ref Latitude,S Ref Longitude,bicycle_aadf,motor_vehicle_aadf
0,51,49.915023,-6.317073,238.2,1082.6
1,52,49.912343,-6.305686,87.4,950.2
2,53,49.917141,-6.306114,181.8,1385.2
3,54,49.917810,-6.298996,61.2,689.6
4,55,49.918585,-6.295094,33.6,352.4


### Creating the dataset of 'danger' locations (with traffic accidents)

In [10]:
def myround(x, base=.0005):
    return base * round(x/base)

In [11]:
# Adding rounded lat and long columns and a grid square column to the London accident dataset
accidents['lat_4dp'] = myround(accidents['Latitude'])
accidents['long_4dp'] = myround(accidents['Longitude'])
accidents['grid_square'] = round(accidents['lat_4dp'],4).map(str) + "," + round(accidents['long_4dp'],4).map(str)

In [12]:
# Get just accidents that are serious/fatal and drop duplicates based on grid_square
serious_fatal_accidents = accidents[accidents['Accident_Severity']!='Slight'].drop_duplicates(['grid_square'])

In [13]:
# Take random sample of 10000 serious/fatal accidents
serious_fatal_accidents_sample = serious_fatal_accidents.sample(n=10000, random_state=42, 
                                                                replace=False).drop_duplicates(['grid_square'])
print(len(serious_fatal_accidents_sample))
serious_fatal_accidents_sample.head()

10000


,Accident_Index,Longitude,Latitude,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,1st_Road_Class,Road_Type,Speed_limit,Junction_Detail,2nd_Road_Class,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Hour,Two_Hour_Groupings,Time_of_Day,Was_Daylight,Was_Bad_Weather,Was_Road_Dry,log_Number_of_Casualties,log_Number_of_Vehicles,LSOA,population_per_hectare,bicycle_aadf,motorbike_aadf,car_aadf,bus_aadf,light_goods_vehicle_aadf,heavy_goods_vehicle_aadf,Road,RCat,lat_4dp,long_4dp,grid_square
1602,201301CW10782,-0.178017,51.511914,Serious,2,1,2013-06-23,Sunday,Unclassified,Single carriageway,30.0,T or staggered junction,Unclassified,Darkness - lights lit,Fine no high winds,Dry,Urban,3,2am-4am,Night,No,No,Yes,0.0,0.693147,E01004694,177.2,1434.4,1700.6,23408.2,1297.4,5069.8,700.4,A402,PA,51.512,-0.1780,"51.512,-0.178"
66680,201501KF60668,0.035556,51.534803,Serious,1,1,2015-09-05,Saturday,B,Single carriageway,30.0,T or staggered junction,Unclassified,Daylight,Fine no high winds,Dry,Urban,19,6pm-8pm,Evening,Yes,No,Yes,0.0,0.000000,E01003491,190.8,29.8,4.2,296.8,0.8,32.8,3.0,U,MCU,51.535,0.0355,"51.535,0.0355"
16842,201301TX20552,-0.373621,51.458227,Serious,2,1,2013-09-20,Friday,A,Single carriageway,30.0,T or staggered junction,Unclassified,Daylight,Fine no high winds,Dry,Urban,16,4pm-6pm,Rush hour,Yes,No,Yes,0.0,0.693147,E01002652,91.5,176.8,146.2,13306.4,383.2,1689.0,363.2,A314,PA,51.458,-0.3735,"51.458,-0.3735"
41291,201401PY20786,0.109107,51.387763,Serious,1,1,2014-10-31,Friday,A,Single carriageway,30.0,Crossroads,C,Daylight,Fine no high winds,Dry,Urban,19,6pm-8pm,Evening,Yes,No,Yes,0.0,0.000000,E01000722,28.3,35.4,292.0,23168.0,633.2,3761.8,1040.0,A224,PA,51.388,0.1090,"51.388,0.109"
87227,2016010000752,-0.095963,51.582192,Serious,1,1,2016-11-08,Tuesday,Unclassified,Single carriageway,20.0,Not at junction or within 20 meters,Not at junction or within 20 meters,Daylight,Fine no high winds,Dry,Urban,9,8am-10am,Morning,Yes,No,Yes,0.0,0.000000,E01002042,60.4,264.8,229.4,13171.8,966.4,2134.8,305.4,A504,PA,51.582,-0.0960,"51.582,-0.096"


In [14]:
# Create model 4 dataset just consisting of traffic and population features
# Keep location features to get safe grid squares later
danger_squares = serious_fatal_accidents_sample[['Latitude', 'Longitude', 'population_per_hectare', 'bicycle_aadf', 
                                'motorbike_aadf', 'car_aadf', 'bus_aadf', 'light_goods_vehicle_aadf', 
                                'heavy_goods_vehicle_aadf', 'lat_4dp', 'long_4dp', 'grid_square']]

In [15]:
# Combine traffic features for motor vehicles like we did in model 1 notebook
to_sum = ['motorbike_aadf', 'car_aadf', 'bus_aadf', 'light_goods_vehicle_aadf', 'heavy_goods_vehicle_aadf']
danger_squares['motor_vehicle_aadf'] = danger_squares[to_sum].sum(axis=1)
danger_squares.drop(to_sum, axis=1, inplace=True)

/home/rsouza/Documents/envs/python_env/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/rsouza/Documents/envs/python_env/lib/python3.6/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
# Getting columns in the right order
danger_squares = danger_squares[['grid_square', 'Latitude', 'Longitude', 'population_per_hectare', 
                                 'bicycle_aadf', 'motor_vehicle_aadf']]

# Rename columns for merging with safe dataset
danger_squares = danger_squares.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})

# Add safe column of all 0's due to being danger dataset
danger_squares['safe'] = 0

In [17]:
# Creating a list of danger squares to download using Google Static Maps API
model4_danger_squares = list(danger_squares.grid_square)

In [18]:
danger_squares.set_index('grid_square', inplace=True)
danger_squares.head()

,latitude,longitude,population_per_hectare,bicycle_aadf,motor_vehicle_aadf,safe
grid_square,,,,,,
"51.512,-0.178",51.511914,-0.178017,177.2,1434.4,32176.4,0
"51.535,0.0355",51.534803,0.035556,190.8,29.8,337.6,0
"51.458,-0.3735",51.458227,-0.373621,91.5,176.8,15888.0,0
"51.388,0.109",51.387763,0.109107,28.3,35.4,28895.0,0
"51.582,-0.096",51.582192,-0.095963,60.4,264.8,16807.8,0


In [19]:
# save dataset for model 4 notebook
#danger_squares.to_csv('model4_danger_dataset.csv')

### Getting the dataset of 'safe' locations (with no accidents)

The same dataset of safe squares will be used from model 3.

In [21]:
# Importing the same safe squares dataset (with no accidents) as was used in model 3
safe_squares = pd.read_csv('../../../data/model3_safe_dataset.csv')
safe_squares.set_index('grid_square', inplace=True)
safe_squares.head()

,latitude,longitude,population_per_hectare,bicycle_aadf,motor_vehicle_aadf,safe
grid_square,,,,,,
"51.572,-0.4435",51.572,-0.4435,31.1,151.8,25847.8,1
"51.407,-0.1835",51.407,-0.1835,142.5,575.0,35016.4,1
"51.468,0.2515",51.468,0.2515,1.5,0.0,128692.0,1
"51.319,-0.286",51.319,-0.2860,4.0,85.2,16208.4,1
"51.341,0.196",51.341,0.1960,2.0,16.4,5928.4,1


### Combining the datasets of safe and danger squares

In [22]:
# Creating the final dataset containing safe and danger squares
df = safe_squares.append(danger_squares)
len(df)

20000

In [23]:
df.drop(['latitude', 'longitude'], axis=1, inplace=True)
df.head()

,population_per_hectare,bicycle_aadf,motor_vehicle_aadf,safe
grid_square,,,,
"51.572,-0.4435",31.1,151.8,25847.8,1
"51.407,-0.1835",142.5,575.0,35016.4,1
"51.468,0.2515",1.5,0.0,128692.0,1
"51.319,-0.286",4.0,85.2,16208.4,1
"51.341,0.196",2.0,16.4,5928.4,1


In [ ]:
#df.to_csv('data/modelling/model4_dataset.csv')

In [24]:
#df = pd.read_csv('data/modelling/model4_dataset.csv', index_col='grid_square')
df.head()

,population_per_hectare,bicycle_aadf,motor_vehicle_aadf,safe
grid_square,,,,
"51.572,-0.4435",31.1,151.8,25847.8,1
"51.407,-0.1835",142.5,575.0,35016.4,1
"51.468,0.2515",1.5,0.0,128692.0,1
"51.319,-0.286",4.0,85.2,16208.4,1
"51.341,0.196",2.0,16.4,5928.4,1


## Example images from the serious/fatal class

The images below show the first five satellite images of areas where serious or fatal accidents occurred. They are primarily large motorways (highways) with multiple lanes, on straight sections of road. This is expected based on the results of the EDA of serious and fatal accidents in notebook 3.

In [25]:
def show_image(image_path, size=[600, 400]):
    img = load_img(image_path, target_size=(size[0], size[1]))
    img_tensor = img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    plt.imshow(img_tensor[0])

In [26]:
imgs_danger_path = 'model4_images/fatal_or_serious/'
imgs_danger = [file for file in os.listdir(imgs_danger_path) if file.endswith('.jpg')]

imgs_danger_plot = []
for img in imgs_danger[:5]:
    imgs_danger_plot.append(os.path.join(imgs_danger_path, img))

FileNotFoundError: [Errno 2] No such file or directory: 'model4_images/fatal_or_serious/'

In [ ]:
fig = plt.figure(figsize=(16,7))
for i, img in enumerate(imgs_danger_plot):
    fig.add_subplot(1,5,i+1)
    plt.suptitle('Example areas where serious accidents occurred', fontsize=15)
    show_image(img)
    plt.axis('off')

## Modelling

### Model iteration 1

The lists of grid squares from the safe and serious square sets were used to download images from the Google Static Maps API using the same code described in notebook 2, into a folder called 'model4_images'.

The same model structure will be used as the best model produced for model 3 (version 3), in order to compare performance.

In [ ]:
# Getting images and reshaping
image_folder = 'model4_images/'
image_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        image_folder, shuffle=False, class_mode='binary',
        target_size=(128, 128), batch_size=20000)

In [ ]:
# Checking the labels
image_generator.class_indices

In [ ]:
images, labels = next(image_generator)

In [ ]:
# Getting the ordered list of filenames for the images
image_files = pd.Series(image_generator.filenames)
image_files = image_files.str.split('/', expand=True)[1].str[:-4]
image_files = list(image_files)

In [ ]:
# Sorting the structured data into the same order as the images
df_sorted = df.reindex(image_files)

Double checking the dataframe is in the same order as the images:

In [ ]:
df_sorted.head(1)

This image is for the correct grid square, confirming that the dataframe is in the correct order.

In [ ]:
plt.imshow(images[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(labels[0]))

The following functions will be used to pre-process the data and create the mixed-input neural network architecture.

In [ ]:
def process_structured_data(df, train, test):
    """
    Pre-processes the given dataframe by minmaxscaling the continuous features (fit-transforming the training data and transforming the test data)
    """
    continuous = ["population_per_hectare", "bicycle_aadf", "motor_vehicle_aadf"]
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    return (trainX, testX)

In [ ]:
def create_mlp(dim, regularizer=None):
    """Creates a simple two-layer MLP with inputs of the given dimension"""
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    model.add(Dense(4, activation="relu", kernel_regularizer=regularizer))
    return model

In [ ]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regularizer=None):
    """
    Creates a CNN with the given input dimension and filter numbers.
    Adapted from the function described here: https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/
    """
    # Initialize the input shape and channel dimension, where the number of channels is the last dimension
    inputShape = (height, width, depth)
    chanDim = -1
 
    # Define the model input
    inputs = Input(shape=inputShape)
 
    # Loop over the number of filters 
    for (i, f) in enumerate(filters):
        # If this is the first CONV layer then set the input appropriately
        if i == 0:
            x = inputs
 
        # Create loops of CONV => RELU => BN => POOL layers
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
    # Final layers - flatten the volume, then Fully-Connected => RELU => BN => DROPOUT
    x = Flatten()(x)
    x = Dense(16, kernel_regularizer=regularizer)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(0.5)(x)
 
    # Apply another fully-connected layer, this one to match the number of nodes coming out of the MLP
    x = Dense(4, kernel_regularizer=regularizer)(x)
    x = Activation("relu")(x)
 
    # Construct the CNN
    model = Model(inputs, x)
 
    # Return the CNN
    return model        

The following functions will be used for evaluation:

In [ ]:
def show_cf(y_true, y_pred, class_names=None, model_name=None):
    """Plots a confusion matrix"""
    cf = confusion_matrix(y_true, y_pred)
    plt.imshow(cf, cmap=plt.cm.Blues)
    
    if model_name:
        plt.title("Confusion Matrix: {}".format(model_name))
    else:
        plt.title("Confusion Matrix")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    class_names = set(y_true)
    tick_marks = np.arange(len(class_names))
    if class_names:
        plt.xticks(tick_marks, class_names)
        plt.yticks(tick_marks, class_names)
    
    thresh = cf.max() / 2.
    
    for i, j in itertools.product(range(cf.shape[0]), range(cf.shape[1])):
        plt.text(j, i, cf[i, j], horizontalalignment='center', color='white' if cf[i, j] > thresh else 'black')

    plt.colorbar()

In [ ]:
def cnn_evaluation(model, history, train_features, train_images, train_labels, test_features, test_images, test_labels, class_names=None, model_name=None):
    """
    Evaluates the performance of a CNN with loss and accuracy plots, a confusion matrix and a classification report for the training and test sets.
    """
    train_acc = history.history['acc']
    val_acc = history.history['val_acc']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    epch = range(1, len(train_acc) + 1)
    plt.plot(epch, train_acc, 'g.', label='Training Accuracy')
    plt.plot(epch, val_acc, 'g', label='Validation acc')
    plt.title('Accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epch, train_loss, 'r.', label='Training loss')
    plt.plot(epch, val_loss, 'r', label='Validation loss')
    plt.title('Loss')
    plt.legend()
    plt.show()
    
    results_test = model.evaluate([test_features, test_images], test_labels)
    print('Test Loss:', results_test[0])
    print('Test Accuracy:', results_test[1])
    
    y_train_pred = np.round(model.predict([train_features, train_images]))
    y_pred = np.round(model.predict([test_features, test_images]))
    
    show_cf(test_labels, y_pred, class_names=class_names, model_name=model_name)
    
    print(classification_report(train_labels, y_train_pred))
    print(classification_report(test_labels, y_pred))

**Conducting the train test split:**

In [ ]:
# Using train_test_split to partition the training and testing structured data attributes and images
(trainAttrX, testAttrX, trainImagesX, testImagesX) = train_test_split(df_sorted, images, test_size=0.25, random_state=42)

In [ ]:
# Setting the labels for y as the safe column
trainY = trainAttrX["safe"]
testY = testAttrX["safe"]

In [ ]:
# Process the structured data
(trainAttrX, testAttrX) = process_structured_data(df_sorted, trainAttrX, testAttrX)

**Building the model:**

In [ ]:
# Create the MLP and CNN models
mlp1 = create_mlp(trainAttrX.shape[1], regularizer=regularizers.l1(0.005))
cnn1 = create_cnn(128, 128, 3, regularizer=regularizers.l1(0.005))
 
# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = concatenate([mlp1.output, cnn1.output])

In [ ]:
print(mlp1.summary())
print(cnn1.summary())

In [ ]:
# The final FC layer head will have two dense layers
x = Dense(4, activation="relu", kernel_regularizer=regularizers.l1(0.005))(combinedInput)
x = Dense(1, activation="sigmoid", kernel_regularizer=regularizers.l1(0.005))(x)

In [ ]:
start = datetime.datetime.now()

model1 = Model(inputs=[mlp1.input, cnn1.input], outputs=x)

# compile the model 
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model1.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=opt)
 
# train the model, and validate with the first 1000 rows of the test set
model1_history = model1.fit([trainAttrX, trainImagesX], trainY, validation_data=([testAttrX[:1000], testImagesX[:1000]], testY[:1000]), epochs=15, batch_size=10)
 
end = datetime.datetime.now()
print("Time taken to run:", end-start)

In [ ]:
model1.save('models/mixed_model4_v1.h5')

In [ ]:
cnn_evaluation(model1, model1_history, trainAttrX, trainImagesX, trainY, testAttrX[1000:], testImagesX[1000:], testY[1000:], class_names=['serious/fatal', 'safe'])

This model is performing well, with an average F1 score of 0.82 (better than the best version of model 3, which scored 0.80). There is no overfitting, and there is a particularly low number of false negatives, which is preferred.

### Examining model predictions

This section uses the best model (iteration 1) to produce a dataframe of predictions for each location in the test set. It also shows some examples of correct model predictions for areas with no accidents and areas with serious/fatal accidents in both rural and urban areas.

In [ ]:
# Loading the model
model1 = load_model('models/mixed_model4_v1.h5')

In [ ]:
# Using the model to make predictions for each of the locations (with images and structured data attributes) in the test set
y_pred = np.round(model1.predict([testAttrX[1000:], testImagesX[1000:]]))

# Reshaping the predictions and turning them into a pandas series
test_predictions = pd.Series(y_pred.reshape((4000,)))
test_predictions = pd.Series(test_predictions)

# Getting a dataframe of the test set locations and actual classes
test_actuals = pd.DataFrame(testY[1000:]).reset_index()

# Making a dataframe of True and Predicted labels so can look up images by grid_square
test_df = pd.concat([test_actuals, test_predictions], axis=1)
test_df.columns = ['grid_square', 'True', 'Predicted']

# Example predictions
test_df[40:60]

**Example images:**

In [ ]:
# Used grid_square column to check urban or rural areas in Google Maps
# Plot examples of safe and fatal/serious images from urban and rural areas to add to presentation
plt.figure(figsize=(12,7))
plt.subplot(2,2,1)
plt.imshow(testImagesX[1000,:,:], cmap='gray')
plt.title('safe')
plt.axis('off')

plt.subplot(2,2,2)
plt.imshow(testImagesX[1008,:,:], cmap='gray')
plt.title('fatal/serious')
plt.axis('off')
plt.suptitle('Rural')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,7))
plt.subplot(2,2,1)
plt.imshow(testImagesX[1029,:,:], cmap='gray')
plt.title('safe')
plt.axis('off')

plt.subplot(2,2,2)
plt.imshow(testImagesX[1041,:,:], cmap='gray')
plt.title('fatal/serious')
plt.axis('off')
plt.suptitle('Urban')
plt.tight_layout()
plt.show()

### Model iteration 2

As one final version, this iteration of the model will add an extra layer to the MLP branch of the neural network, as it is currently only one small layer and an output layer. It will also be run for slightly more epochs.

In [ ]:
# Getting images and reshaping
image_folder = 'model4_images/'
image_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        image_folder, shuffle=False, class_mode='binary',
        target_size=(128, 128), batch_size=20000)

In [ ]:
# Checking the labels
image_generator.class_indices

In [ ]:
images, labels = next(image_generator)

In [ ]:
# Getting the ordered list of filenames for the images
image_files = pd.Series(image_generator.filenames)
image_files = image_files.str.split("\\", expand=True)[1].str[:-4]
image_files = list(image_files)

In [ ]:
# Sorting the structured data into the same order as the images
df_sorted = df.reindex(image_files)

In [ ]:
# Adding an extra layer to the 'create MLP' function
def create_mlp(dim, regularizer=None):
    """Creates a three-layer MLP with inputs of the given dimension"""
    model = Sequential()
    model.add(Dense(16, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    model.add(Dense(32, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    model.add(Dense(4, activation="relu", kernel_regularizer=regularizer))
    return model

In [ ]:
# Create the MLP and CNN models
mlp2 = create_mlp(trainAttrX.shape[1], regularizer=regularizers.l1(0.005))
cnn2 = create_cnn(128, 128, 3, regularizer=regularizers.l1(0.005))
 
# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = concatenate([mlp2.output, cnn2.output])

In [ ]:
print(mlp2.summary())
print(cnn2.summary())

In [ ]:
# The final FC layer head will have two dense layers
x = Dense(4, activation="relu", kernel_regularizer=regularizers.l1(0.005))(combinedInput)
x = Dense(1, activation="sigmoid", kernel_regularizer=regularizers.l1(0.005))(x)

In [ ]:
model2 = Model(inputs=[mlp2.input, cnn2.input], outputs=x)

In [ ]:
# Visualising the model
SVG(model_to_dot(model2, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
plot_model(model2, to_file='mixed_model4_v2.png')

In [ ]:
start = datetime.datetime.now()

# compile the model 
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model2.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=opt)
 
# train the model, and validate with the first 1000 rows of the test set
model2_history = model2.fit([trainAttrX, trainImagesX], trainY, validation_data=([testAttrX[:1000], testImagesX[:1000]], testY[:1000]), epochs=20, batch_size=10)
 
end = datetime.datetime.now()
print("Time taken to run:", end-start)

In [ ]:
model2.save('models/mixed_model4_v2.h5')

In [ ]:
cnn_evaluation(model2, model2_history, trainAttrX, trainImagesX, trainY, testAttrX[1000:], testImagesX[1000:], testY[1000:], class_names=['serious', 'safe'])

This model performs slightly worse than the first version, on the basis of both the F1 score and the recall rate/proportion of false negatives. Therefore, the preferred model is version 1.

## Summary and potential directions for future work

The best model produced was version 1, which had an average F1 score of 0.82 and had no overfitting.

To try and improve the accuracy of the model, some of the following could be tried in further model iterations:
- Increasing the image size for input into the CNN branch
- Using a pre-trained model as the base for the CNN (which increased accuracy in model 2)
- Parameter tuning of both the CNN and MLP components (e.g. experimenting with regularization and optimization)
- Parameter tuning of the final fully-connected layer head
- Adding additional data to the structured dataset about the local area
- Limiting the 'safe square' part of the dataset to only squares that contain roads of some type, i.e. excluding fields, forests and bodies of water